In [8]:
"""Бот, поясняющий некоторые слова современного молодежного сленга"""
import pandas as pd
from nltk.metrics.distance import edit_distance

In [ ]:
df = pd.read_excel('slang list.xlsx')

In [3]:
bot_dic = dict(zip(df['Слово'], df['Значение']))

In [66]:
def find_sim(word): # функция для нахождения похожего слова в словаре
    if len(word) <= 4: # задаем предельное расстояние преобразования между словами, которое нас устроит
        lim_distance = 1
    else:    
        lim_distance = int(len(word)*0.5)
    resword = ''
    distance = len(word)
    for key in bot_dic.keys():
        key = key.lower()
        if edit_distance(word, key, transpositions=True) <= lim_distance:
            distance = edit_distance(word, key, transpositions=True)
            resword = key.capitalize()
    if distance <= lim_distance:
        return resword, bot_dic[resword]

In [67]:
def running_bot(inp): # берем слово из сообщения пользователя и возвращаем результат поиска либо сообщение о том, что ничего не найдено
    word = inp.capitalize() 
    if word in bot_dic.keys(): # если в словаре есть точное совпадение 
        return bot_dic[word]    
    else:
        word = word.lower()
        res = find_sim(word) # если нет, возвращает похожее
        if res is None:
            return "Ничего не найдено"
        else:
            return f"Возможно, вы искали: {res[0]}\nЗначение: {res[1]}"
        
   
    

In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, CallbackContext, Filters

def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text(f'Введите слово')


def help_command(update, context):
    update.message.reply_text('Этот бот подскажет вам значения слов современного молодежного сленга. Вводите по одному слову.')


def bot_answer(update, context):
    user_input = update.message.text
    answer = running_bot(user_input)
    update.message.reply_text(answer)


def main():
    updater = Updater("token_dummy", use_context=True)

    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, bot_answer))

    updater.start_polling()
    updater.idle()

In [ ]:
main()